In [1]:
import os
import json
from pathlib import Path
# os.environ['LMDB_FORCE_CFFI'] = '1'
# # CFFI variant is loaded.
import lmdb
from txyl_common.biocyc_facade.pgdb import Pgdb, Dat

In [2]:
pgdb_dir = Path("/home/tony/workspace/grad/cpsc445-project/data/txyl_local/biocyc_facade_pgdbs")

pmeta = Pgdb(pgdb_dir.joinpath("meta26_0.1_2"))
emeta = Pgdb(pgdb_dir.joinpath("eco26_0.1_2"))

In [3]:
env = lmdb.open("./cache/biocyc_test.lmdb", map_size=16*1024**3)

with env.begin(write=True) as txn:
    ENC = "utf-8"
    for d in Dat:
        table = pmeta.GetDataTable(d)
        for k, v in table.items():
            txn.put(
                k.encode(ENC),
                json.dumps(v, separators=(",",":")).encode(ENC)
            )

    for d in Dat:
        table = emeta.GetDataTable(d)
        for k, v in table.items():
            txn.put(
                k.encode(ENC),
                json.dumps(v, separators=(",",":")).encode(ENC)
            )
env.close()

In [4]:
env = lmdb.open("./cache/meta_test.lmdb", map_size=16*1024**3)

with env.begin(write=True) as txn:
    ENC = "utf-8"
    for d in Dat:
        table = pmeta.GetDataTable(d)
        for k, v in table.items():
            txn.put(
                k.encode(ENC),
                json.dumps(v, separators=(",",":")).encode(ENC)
            )
env.close()

In [5]:
env = lmdb.open("./cache/eco_test.lmdb", map_size=16*1024**3, max_dbs=16)

ENC = "utf-8"
for d in Dat:
    namespace = env.open_db(d.name.encode(ENC), create=True)
    with env.begin(write=True, db=namespace) as txn:
        table = emeta.GetDataTable(d)
        for k, v in table.items():
            txn.put(
                k.encode(ENC),
                json.dumps(v, separators=(",",":")).encode(ENC)
            )
env.close()

In [2]:
env = lmdb.open("./cache/eco_test.lmdb", map_size=16*1024**3, max_dbs=16)
ENC = "utf-8"
rtable = env.open_db(Dat.REACTIONS.name.encode(ENC))

all_entries = []
try:
    with env.begin() as txn:
        ENC = "utf-8"
        # cur = txn.cursor()
        cur = txn.cursor(rtable)
        for k, v in cur:
            all_entries.append((
                k.decode(ENC),
                json.loads(v.decode(ENC))
            ))
finally:
    env.close()
len(all_entries)

3052

In [ ]:
env = lmdb.open("./cache/eco_test.lmdb", map_size=16*1024**3, max_dbs=16)
ENC = "utf-8"
rtable = env.open_db(Dat.REACTIONS.name.encode(ENC))

all_entries = []
try:
    with env.begin() as txn:
        ENC = "utf-8"
        # cur = txn.cursor()
        cur = txn.cursor(rtable)
        for k, v in cur:
            print(k.decode(ENC), v.decode("utf-8"))
            # break
            # all_entries.append((
            #     k.decode(ENC),
            #     json.loads(v.decode(ENC))
            # ))
finally:
    env.close()
len(all_entries)

In [10]:
env.close()

In [8]:
env = lmdb.open("./cache/lmdb_test.lmdb", map_size=16*1024**3)

In [9]:
with env.begin() as txn:
    cursor = txn.cursor()
    for k, v in cursor:
        print(k, v)

b'test_key' b'{"x":1,"b":"Asdf","inner":{"ix":2},"array":[1,2,3,4]}'


In [3]:
test_data = {
    "x": 1,
    "b": "Asdf",
    "inner": {
        "ix": 2,
    },
    "array": [1, 2, 3, 4]
}

with env.begin(write=True) as transaction:
    sdata = json.dumps(test_data, separators=(',',':'))
    bdata = sdata.encode("utf-8")
    key = "test_key"
    bkey = key.encode("utf-8")
    transaction.put(bkey, bdata)

In [4]:
with env.begin() as transaction:
    _bdata: bytes = transaction.get(bkey)
    out = json.loads(_bdata.decode("utf-8"))

In [5]:
out

{'x': 1, 'b': 'Asdf', 'inner': {'ix': 2}, 'array': [1, 2, 3, 4]}

In [17]:
env.close()